# Preprocessing Script

In [1]:
import os
import sys
import importlib
from importlib import reload 
from dataclasses import dataclass, field, fields
from itertools import compress
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import scipy
import scipy.io as sio
from scipy import signal
from scipy.signal import spectrogram, butter, filtfilt, freqz
try:
    from scipy.signal import hann
except:
    from scipy.signal.windows import hann
from scipy.integrate import simps
from scipy import stats

import pickle
import json
import csv

#mne
import mne_bids
import mne
from mne.time_frequency import tfr_morlet 
from mne.stats import permutation_cluster_test
from mne.preprocessing import ICA, create_ecg_epochs


In [2]:
from PerceiveImport.classes import (
    main_class, modality_class, metadata_class,
    session_class, condition_class, task_class,
    contact_class, run_class
)

import PerceiveImport.methods.load_rawfile as load_rawfile
import PerceiveImport.methods.find_folders as PyPerceive_find_folders
import PerceiveImport.methods.metadata_helpers as metaHelpers


# utils
import beta_profile.utils.find_folders as find_folders
import beta_profile.utils.io as io

# tfr
import beta_profile.beta_profile.tfr_preprocessing as tfr

# plots
import beta_profile.beta_profile.power_spectra_plots as power_spectra


importlib.reload(find_folders)
importlib.reload(io)
importlib.reload(tfr)
importlib.reload(power_spectra)


<module 'beta_profile.beta_profile.power_spectra_plots' from '/Users/jenniferbehnke/code/Beta_profile_project/Beta_profile/src/beta_profile/beta_profile/power_spectra_plots.py'>

In [7]:
find_folders.get_onedrive_path()

'/Users/jenniferbehnke/Charité - Universitätsmedizin Berlin/AG Bewegungsstörungen - Percept - Percept_Data_structured'

### Load MNE Objects from PyPerceive

In [19]:
loaded_data = io.load_py_perceive_object(
    sub="017",
    session="fu3m",
    condition="m0s0",
    hemisphere="Right"
)
loaded_data


	### WARNING: NaNs in Metadata Table sub-017 ###
NaNs in: sub-017_ses-2021040709594199_run-CHRONIC20210404124649.mat
NaNs in: sub-017_ses-2021040809491499_run-CHRONIC20210407065701.mat
NaNs in: sub-017_ses-2021041212054199_run-CHRONIC20210408063010.mat
NaNs in: sub-017_ses-2021041410364999_run-CHRONIC20210412103008.mat
NaNs in: sub-017_ses-2021041410364999_run-IS20210414104500.mat
NaNs in: sub-017_ses-2021041410364999_run-IS20210414104900.mat
NaNs in: sub-017_ses-2021041410364999_run-IS20210414105100.mat
NaNs in: sub-017_ses-2021041410364999_run-IS20210414105600.mat
NaNs in: sub-017_ses-2021041410364999_run-IS20210414105900.mat
NaNs in: sub-017_ses-2021041410364999_run-IS20210414110100.mat
NaNs in: sub-017_ses-2021041410364999_run-IS20210414110400.mat
NaNs in: sub-017_ses-2021041410364999_run-IS20210414110700.mat
NaNs in: sub-017_ses-2021041410364999_run-IS20210414110800.mat
NaNs in: sub-017_ses-2021041410364999_run-IS20210414111200.mat
NaNs in: sub-017_ses-2021041410364999_run-IS2021

PerceiveData(sub='017', incl_modalities=['survey'], incl_session=['fu3m'], incl_condition=['m0s0'], incl_task=['rest'], incl_contact=['RingR', 'SegmIntraR', 'SegmInterR'], import_json=False, warn_for_metaNaNs=True, use_chronic_json_file=True, allow_NaNs_in_metadata=False)

In [22]:
data = io.extract_data_from_py_perceive(
    sub="017",
    session="fu3m",
    condition="m0s0",
    hemisphere="Right"
)
data


	### WARNING: NaNs in Metadata Table sub-017 ###
NaNs in: sub-017_ses-2021040709594199_run-CHRONIC20210404124649.mat
NaNs in: sub-017_ses-2021040809491499_run-CHRONIC20210407065701.mat
NaNs in: sub-017_ses-2021041212054199_run-CHRONIC20210408063010.mat
NaNs in: sub-017_ses-2021041410364999_run-CHRONIC20210412103008.mat
NaNs in: sub-017_ses-2021041410364999_run-IS20210414104500.mat
NaNs in: sub-017_ses-2021041410364999_run-IS20210414104900.mat
NaNs in: sub-017_ses-2021041410364999_run-IS20210414105100.mat
NaNs in: sub-017_ses-2021041410364999_run-IS20210414105600.mat
NaNs in: sub-017_ses-2021041410364999_run-IS20210414105900.mat
NaNs in: sub-017_ses-2021041410364999_run-IS20210414110100.mat
NaNs in: sub-017_ses-2021041410364999_run-IS20210414110400.mat
NaNs in: sub-017_ses-2021041410364999_run-IS20210414110700.mat
NaNs in: sub-017_ses-2021041410364999_run-IS20210414110800.mat
NaNs in: sub-017_ses-2021041410364999_run-IS20210414111200.mat
NaNs in: sub-017_ses-2021041410364999_run-IS2021

{'RingR': <RawArray | 6 x 5288 (21.1 s), ~257 kB, data loaded>,
 'SegmIntraR': <RawArray | 6 x 5288 (21.1 s), ~257 kB, data loaded>,
 'SegmInterR': <RawArray | 3 x 5312 (21.2 s), ~132 kB, data loaded>}

In [23]:
ring_data = data["SegmIntraR"]

Pick a channel of interest, perform a Fourier Transform and then plot the PSD

In [24]:
structured_signals = tfr.pick_channels_of_interest(
    sub="017",
    session="fu3m",
    condition="m0s0",
    hemisphere="Right"
)


	### WARNING: NaNs in Metadata Table sub-017 ###
NaNs in: sub-017_ses-2021040709594199_run-CHRONIC20210404124649.mat
NaNs in: sub-017_ses-2021040809491499_run-CHRONIC20210407065701.mat
NaNs in: sub-017_ses-2021041212054199_run-CHRONIC20210408063010.mat
NaNs in: sub-017_ses-2021041410364999_run-CHRONIC20210412103008.mat
NaNs in: sub-017_ses-2021041410364999_run-IS20210414104500.mat
NaNs in: sub-017_ses-2021041410364999_run-IS20210414104900.mat
NaNs in: sub-017_ses-2021041410364999_run-IS20210414105100.mat
NaNs in: sub-017_ses-2021041410364999_run-IS20210414105600.mat
NaNs in: sub-017_ses-2021041410364999_run-IS20210414105900.mat
NaNs in: sub-017_ses-2021041410364999_run-IS20210414110100.mat
NaNs in: sub-017_ses-2021041410364999_run-IS20210414110400.mat
NaNs in: sub-017_ses-2021041410364999_run-IS20210414110700.mat
NaNs in: sub-017_ses-2021041410364999_run-IS20210414110800.mat
NaNs in: sub-017_ses-2021041410364999_run-IS20210414111200.mat
NaNs in: sub-017_ses-2021041410364999_run-IS2021

In [25]:
structured_signals

,original_channel,channel,lfp,lfp_group
0,LFP_R_12_STN_MT,12,"[-1.246763548, 0.2266842814, 2.606869236, 2.26...",RingR
1,LFP_R_01_STN_MT,01,"[-0.786311101, 1.684952359, 1.347961887, -4.38...",RingR
2,LFP_R_23_STN_MT,23,"[-6.945155064, -7.514430069, -10.5885151, -11....",RingR
3,LFP_R_1A1B_STN_MT,1A1B,"[3.819225348, 2.583593618, -0.3369904719, -0.4...",SegmIntraR
4,LFP_R_1B1C_STN_MT,1B1C,"[9.677675089, 10.4746601, 8.539125079, 6.14817...",SegmIntraR
5,LFP_R_1A1C_STN_MT,1A1C,"[13.89031013, 14.45958513, 9.905385091, 6.3758...",SegmIntraR
6,LFP_R_2A2B_STN_MT,2A2B,"[3.626948502, 3.40026422, 3.966974924, 1.02007...",SegmIntraR
7,LFP_R_2B2C_STN_MT,2B2C,"[-2.920584089, -3.70689519, -4.830196763, -7.1...",SegmIntraR
8,LFP_R_2A2C_STN_MT,2A2C,"[3.415650031, -1.593970015, -3.30179503, -6.03...",SegmIntraR
9,LFP_R_1A2A_STN_MT,1A2A,"[6.964469752, 7.413790381, 1.572622202, 2.2466...",SegmInterR


In [10]:
lfp = structured_signals.lfp.values[0]

filtered_lfp = tfr.band_pass_filter(lfp)
fourier_transformed_lfp = tfr.fourier_transform(filtered_lfp)

f = fourier_transformed_lfp["freq"]
psd = fourier_transformed_lfp["average_Sxx"]

plt.plot(f, psd)

In [11]:
peaks = scipy.signal.find_peaks(psd, height=0.03)
peaks

(array([ 7,  9, 16, 19, 21, 27, 34, 37]),
 {'peak_heights': array([0.68151314, 0.61371891, 0.2517362 , 0.19353705, 0.18275293,
         0.09339423, 0.06620088, 0.0494112 ])})

In [12]:
peaks_height = peaks[1]["peak_heights"] # np.array of y-value of peaks = power
peaks_pos = f[peaks[0]] # np.array of indeces on x-axis of peaks = frequency

peak_pos = peaks_pos[(peaks_pos >= 13) & (peaks_pos <= 35)]
peak_height = peaks_height[(peaks_pos >= 13) & (peaks_pos <= 35)] 

In [13]:
peak_pos

array([16., 19., 21., 27., 34.])

In [14]:
peak_height

array([0.2517362 , 0.19353705, 0.18275293, 0.09339423, 0.06620088])

In [15]:
highest_peak_height = peak_height.max()
# get the index of the highest peak y value to get the corresponding peak position x
ix = np.where(peak_height == highest_peak_height)
highest_peak_pos = peak_pos[ix].item()

highest_peak_pos = int(highest_peak_pos) # CF als integer

# 2) go -2 and +3 indeces 
index_low_cut = highest_peak_pos-2
index_high_cut = highest_peak_pos+3

index_low_cut, index_high_cut

cf_4Hz_range = np.arange(index_low_cut, index_high_cut, 1)
cf_4Hz_range

array([14, 15, 16, 17, 18])

In [16]:
# calculate area under the curve of power
beta_power_in_freq_range = psd[
    index_low_cut : index_high_cut
]  # select the power values by indexing from frequency range first until last value

beta_power_area_under_curve = simps(beta_power_in_freq_range, cf_4Hz_range)
beta_power_area_under_curve

/var/folders/8d/3m_ljz890_j62q7hb7mpzdc40000gn/T/ipykernel_57913/661750252.py:6: DeprecationWarning: 'scipy.integrate.simps' is deprecated in favour of 'scipy.integrate.simpson' and will be removed in SciPy 1.14.0
  beta_power_area_under_curve = simps(beta_power_in_freq_range, cf_4Hz_range)


0.8820567733607397

In [17]:
beta_power_in_freq_range


array([0.19360024, 0.23087969, 0.2517362 , 0.2101268 , 0.1850717 ])

### Filter and FT of LFP

In [11]:
beta_profile = tfr.main_tfr(
    sub="017",
    session="fu3m",
    condition="m0s0",
    hemisphere="Right"
)


	### WARNING: NaNs in Metadata Table sub-017 ###
NaNs in: sub-017_ses-2021040709594199_run-CHRONIC20210404124649.mat
NaNs in: sub-017_ses-2021040809491499_run-CHRONIC20210407065701.mat
NaNs in: sub-017_ses-2021041212054199_run-CHRONIC20210408063010.mat
NaNs in: sub-017_ses-2021041410364999_run-CHRONIC20210412103008.mat
NaNs in: sub-017_ses-2021041410364999_run-IS20210414104500.mat
NaNs in: sub-017_ses-2021041410364999_run-IS20210414104900.mat
NaNs in: sub-017_ses-2021041410364999_run-IS20210414105100.mat
NaNs in: sub-017_ses-2021041410364999_run-IS20210414105600.mat
NaNs in: sub-017_ses-2021041410364999_run-IS20210414105900.mat
NaNs in: sub-017_ses-2021041410364999_run-IS20210414110100.mat
NaNs in: sub-017_ses-2021041410364999_run-IS20210414110400.mat
NaNs in: sub-017_ses-2021041410364999_run-IS20210414110700.mat
NaNs in: sub-017_ses-2021041410364999_run-IS20210414110800.mat
NaNs in: sub-017_ses-2021041410364999_run-IS20210414111200.mat
NaNs in: sub-017_ses-2021041410364999_run-IS2021

In [4]:
# freq range info: peak CF, peak power, average in f band
beta_profile[0]

,channel,f_range,power_in_f_range,peak_CF,peak_power,peak_4Hz_power
0,01,low_beta,0.149053,16,0.197216,0.649924
1,01,high_beta,0.124031,30,0.156397,0.551086
2,01,beta,0.132734,16,0.197216,0.649924
3,12,low_beta,0.167254,20,0.180626,0.696192
4,12,high_beta,0.385218,30,0.930798,2.991647
5,12,beta,0.309404,30,0.930798,2.991647
6,23,low_beta,0.274882,14,0.423399,1.367244
7,23,high_beta,0.325524,30,0.759449,2.278661
8,23,beta,0.307910,30,0.759449,2.278661
9,02,low_beta,0.294236,14,0.355850,1.331224


In [5]:
# LFP and PSD of channels
beta_profile[1]

,channel,unfiltered_lfp,filtered_lfp,frequencies,filtered_psd
0,01,"[-0.786311101, 1.684952359, 1.347961887, -4.38...","[-0.30758105136976777, 2.2056769476972526, -0....","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0036195225034, 0.0015774788021761979, 5.073..."
1,12,"[-1.246763548, 0.2266842814, 2.606869236, 2.26...","[-0.08327418749728867, 1.2584941488995642, 3.4...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0016124214796523597, 0.0007188852363214915,..."
2,23,"[-6.945155064, -7.514430069, -10.5885151, -11....","[0.2940421246311222, -0.017638275702081374, -3...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0012702411046273965, 0.0005747889429077157,..."
3,02,"[-2.732520025, 2.504810023, 7.059010065, 0.796...","[-0.3916923912617303, 4.377072973223199, 7.082...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0034835006509015554, 0.0015695553706329547,..."
4,13,"[-8.083705074, -7.514430069, -9.336110086, -10...","[0.08305150720339932, 0.3641325006540761, -1.1...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.001601346144628898, 0.0006850434071054357, ..."
5,1A1B,"[3.819225348, 2.583593618, -0.3369904719, -0.4...","[0.07377297607353592, -1.247369249839861, -3.8...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0020800414125280056, 0.0009128615584494922,..."
6,1A1C,"[13.89031013, 14.45958513, 9.905385091, 6.3758...","[0.14689685996013893, 1.4918303838805476, -3.0...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.002135140386532191, 0.0009421243147289936, ..."
7,1B1C,"[9.677675089, 10.4746601, 8.539125079, 6.14817...","[0.17513463789303502, 1.5084412363959525, 0.05...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0023168511293123217, 0.0010299513910160354,..."
8,2A2B,"[3.626948502, 3.40026422, 3.966974924, 1.02007...","[-0.3870251376976327, -0.31591007423474726, 0....","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0014877637582286782, 0.0006709156328947732,..."
9,2A2C,"[3.415650031, -1.593970015, -3.30179503, -6.03...","[-0.16734944059365583, -4.688051512430658, -5....","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0018866380501464971, 0.0008361019304816873,..."


In [8]:
beta_profile[1]

(5288,)

## Save time series and power spectral data as pickle files

In [32]:
# band-pass filtered 5-95 Hz
# fourier transform: 1Hz resolution, 1 sec window length
HEMISPHERES = ["Right", "Left"]
FREQ_BANDS = ["beta", "high_beta", "low_beta"]

sub_list = ["030",]

for sub in sub_list:

    for hem in HEMISPHERES:

        data = io.save_tfr_and_psd_to_pickle(
            sub=sub,
            session="fu3m",
            condition="m0s0",
            hemisphere=hem
        )

# 3mfu: "017", "019", "021", "024", "025", "029", "030", "031", "032", "033", "033", "040", "041", "045", "050", "059", "060", "061", "062", "063", "065", "066" 
# 12mfu: "021", "028", "036", "047", "048", "049", "052", "055",


	### WARNING: NaNs in Metadata Table sub-030 ###
NaNs in: sub--20210928PStn_ses-2022011207372293_run-BSTD20220112075200.mat
NaNs in: sub--20210928PStn_ses-2022011207372293_run-BSTD20220112080400.mat
NaNs in: sub--20210928PStn_ses-2022011207372293_run-BSTD20220112081600.mat
NaNs in: sub--20210928PStn_ses-2022011207372293_run-BSTD20220112082700.mat
NaNs in: sub--20210928PStn_ses-2022011207372293_run-BSTD20220112084000.mat
NaNs in: sub--20210928PStn_ses-2022011207372293_run-BSTD20220112084700.mat
NaNs in: sub--20210928PStn_ses-2022011207372293_run-BSTD20220112085800.mat
NaNs in: sub--20210928PStn_ses-2022011207372293_run-BSTD20220112091000.mat
NaNs in: sub--20210928PStn_ses-2022011207372293_run-BSTD20220112092200.mat
NaNs in: sub--20210928PStn_ses-2022011207372293_run-BSTD20220112093400.mat
NaNs in: sub-030_ses-2022011210021793_run-BSTD20220112100500.mat
NaNs in: sub-030_ses-2022011210021793_run-BSTD20220112101500.mat
NaNs in: sub-030_ses-2022011210021793_run-BSTD20220112102700.mat
NaNs 

In [26]:
data[1]

,channel,unfiltered_lfp,filtered_lfp,frequencies,filtered_psd
0,01,"[-0.786311101, 1.684952359, 1.347961887, -4.38...","[-0.30758105136976777, 2.2056769476972526, -0....","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0036195225034, 0.0015774788021761979, 5.073..."
1,12,"[-1.246763548, 0.2266842814, 2.606869236, 2.26...","[-0.08327418749728867, 1.2584941488995642, 3.4...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0016124214796523597, 0.0007188852363214915,..."
2,23,"[-6.945155064, -7.514430069, -10.5885151, -11....","[0.2940421246311222, -0.017638275702081374, -3...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0012702411046273965, 0.0005747889429077157,..."
3,02,"[-2.732520025, 2.504810023, 7.059010065, 0.796...","[-0.3916923912617303, 4.377072973223199, 7.082...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0034835006509015554, 0.0015695553706329547,..."
4,13,"[-8.083705074, -7.514430069, -9.336110086, -10...","[0.08305150720339932, 0.3641325006540761, -1.1...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.001601346144628898, 0.0006850434071054357, ..."
5,03,"[-8.986412583, -4.830196763, -6.964469752, -11...","[-0.17769358989646147, 3.4271125189413207, 0.1...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0048286096717373266, 0.0021499478922394743,..."
6,1A1B,"[3.819225348, 2.583593618, -0.3369904719, -0.4...","[0.07377297607353592, -1.247369249839861, -3.8...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0020800414125280056, 0.0009128615584494922,..."
7,1A1C,"[13.89031013, 14.45958513, 9.905385091, 6.3758...","[0.14689685996013893, 1.4918303838805476, -3.0...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.002135140386532191, 0.0009421243147289936, ..."
8,1B1C,"[9.677675089, 10.4746601, 8.539125079, 6.14817...","[0.17513463789303502, 1.5084412363959525, 0.05...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0023168511293123217, 0.0010299513910160354,..."
9,2A2B,"[3.626948502, 3.40026422, 3.966974924, 1.02007...","[-0.3870251376976327, -0.31591007423474726, 0....","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0014877637582286782, 0.0006709156328947732,..."


In [34]:
import pickle

# Specify the path to your .pkl file
sub_path = io.check_or_create_sub_path(sub="019")
file_path = os.path.join(sub_path, "time_series_and_power_details_sub-019_hem-Right_ses-fu3m_cond-m0s0.pkl")

# Open the file and load the data
with open(file_path, 'rb') as file:
    data = pickle.load(file)

# Now, data contains the contents of the .pkl file
data

,channel,unfiltered_lfp,filtered_lfp,frequencies,filtered_psd
0,01,"[-10.88084551, -10.20079266, -6.57384416, 0.34...","[-0.4589451633010853, -0.580505915837762, 2.78...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.004047373924052294, 0.0018691093327629933, ..."
1,12,"[-1.029352714, -2.744940571, -5.604253665, -2....","[0.1736159801239046, -1.7395147157663369, -4.0...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0019439901763742388, 0.0008787828005157202,..."
2,23,"[-0.5667107034, -4.420343487, -7.027212722, -6...","[0.4092841530735279, -3.8957294268991713, -6.0...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0012551199936852258, 0.0005480967422086922,..."
3,02,"[-11.38550011, -13.66260013, -13.43489012, -2....","[-0.287475614439134, -3.606811011413838, -2.87...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0047866058441783845, 0.002239494813986488, ..."
4,13,"[-3.659920761, -6.519233856, -8.921056855, -8....","[0.49708939627744153, -2.7610407584092704, -5....","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.003211861869782908, 0.0014031274462448452, ..."
5,03,"[-14.01716228, -17.11932114, -16.88953159, -8....","[0.0648703316537032, -4.132547741398584, -4.62...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.004997773452922377, 0.0022536480057861225, ..."
6,1A1B,"[-1.493632046, -1.838316364, -1.838316364, -1....","[-0.03242462244386463, -0.19739556009685785, -...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0021562947776363087, 0.0009770868780022725,..."
7,1A1C,"[9.79153009, 6.148170057, 3.757215035, 2.73252...","[-0.21777695487015533, -3.497413744973602, -5....","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0009608932404881947, 0.00041911582949327877..."
8,1B1C,"[9.721664521, 6.633606379, 4.117410856, 2.8593...","[-0.348501588052868, -3.1813703501311887, -5.0...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.001850285603577023, 0.000816376506998312, 2..."
9,2A2B,"[3.31680319, 2.401822999, 2.058705428, 1.14372...","[0.44883409454373147, 0.1573419597201467, 0.16...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.002956486482318122, 0.001295597051887143, 4..."
